In [2]:
import pickle 
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer
from datasets import load_from_disk 
import os
import sys
sys.path.insert(0, '/zhome/a6/6/127219/Speciale/master_project')
from src.visualization.tsne_visual import visualize_layerwise_embeddings#, visualize_one_layer
from src.models.transformers_modeling_roberta import RobertaForSequenceClassification_Linear, RobertaForSequenceClassification_Original


In [3]:
# checkpoint = "/work3/s174498/final/test_original_head/checkpoint-1500"
checkpoint = "/work3/s174498/final/linear_head/checkpoint-1500"

tokenizer = RobertaTokenizer.from_pretrained(checkpoint)
tokenizer.model_max_len=512
model = RobertaForSequenceClassification_Linear.from_pretrained(checkpoint, output_hidden_states = True, return_dict = True)

In [4]:
# tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
# model = RobertaForSequenceClassification.from_pretrained('roberta-base',output_hidden_states = True,return_dict = True)
def preprocess_function(examples):
    return tokenizer(examples["sentence"], truncation=True)

In [5]:
datadir = '/work3/s174498/sst2_dataset/'
test_dataset = load_from_disk(datadir + 'test_dataset')

In [6]:
datadir = '/work3/s174498/concept_random_dataset/wikipedia_20220301/gender_concepts/woman_female'
ds_woman = load_from_disk(datadir)
ds_woman = ds_woman.remove_columns(['title'])
ds_woman = ds_woman.rename_column('text_list','sentence')
ds_woman = ds_woman.add_column('label',[1]*len(ds_woman))
ds_woman = ds_woman.add_column('idx',list(range(len(ds_woman))))
ds_woman

Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 294
})

In [8]:
import random

In [14]:
datadir = '/work3/s174498/concept_random_dataset/wikipedia_split'
ds_random = load_from_disk(datadir)
ds_random = ds_random.remove_columns(['simple_sentence_1','simple_sentence_2'])
ds_random = ds_random.rename_column('complex_sentence','sentence')
ds_random = ds_random.add_column('label',[0]*len(ds_random))
ds_random = ds_random.add_column('idx',list(range(0,len(ds_random))))
#ds_random2 = ds_random.add_column('idx',list(range(0,len(ds_random))))
ds_random = ds_random.filter(lambda example, idx: idx <200000, with_indices=True)

  0%|          | 0/990 [00:00<?, ?ba/s]

In [15]:
#start_with_ar = 
ds_random_1 = ds_random.filter(lambda example, idx: idx in random.sample(range(0,len(ds_random)), len(ds_woman)), with_indices=True)


  0%|          | 0/200 [00:00<?, ?ba/s]

In [16]:
ds_random_2 = ds_random.filter(lambda example, idx: idx in random.sample(range(0,len(ds_random)), len(ds_woman)), with_indices=True)


Loading cached processed dataset at /work3/s174498/concept_random_dataset/wikipedia_split/cache-cf3ed8be676c16e8.arrow


In [18]:
from datasets import concatenate_datasets

ds = concatenate_datasets([ds_woman, ds_random_1,ds_random_2])
ds

Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 810
})

In [1]:
ds['label']

NameError: name 'ds' is not defined

In [12]:
tokenized_test = ds.map(preprocess_function, batched=True)


  0%|          | 0/1 [00:00<?, ?ba/s]

#### If wanting to change features 

In [ ]:
from datasets import ClassLabel, Value
features = ds_random.features.copy()
features['label']  = Value('int64')
features['idx']  = Value('int64')
ds_woman = ds_woman.cast(features)
ds_woman.features

In [ ]:

trainer = Trainer(
    model=model,                        
    tokenizer=tokenizer
)

output = trainer.predict(tokenized_test)
with open(f'/work3/s174498/roberta_files/output_roberta_linear_woman_random.pickle', 'wb') as handle:
    pickle.dump(output, handle, protocol=pickle.HIGHEST_PROTOCOL)

# avg. runtime 2 min

### Load data

In [13]:
with open('/work3/s174498/roberta_files/output_roberta_linear_woman_random.pickle', 'rb') as handle:
    output_linear = pickle.load(handle)

In [14]:
# fine tune linear 
labels = output_linear.label_ids
hidden_states = output_linear.predictions[1][1:]
title = 'linear_results'
layers_to_visualize = [0,1,2,3,4,5,6,7,8,9,10,11]
init = 'pca'
labels_text = ['random','woman']

In [16]:
perplexity = 10
visualize_layerwise_embeddings(hidden_states,labels,title,layers_to_visualize, perplexity, init , NEW_VAR = labels_text, mean = False, save = False)

TypeError: visualize_layerwise_embeddings() got an unexpected keyword argument 'NEW_VAR'

In [7]:
visualize_layerwise_embeddings()   #(hidden_states,labels,title,layers_to_visualize, perplexity, init, labels_text = ['random','woman'], mean = False, save = False)

TypeError: visualize_layerwise_embeddings() got an unexpected keyword argument 'labels_text'